# All imports necessary

In [ ]:
import os

In [ ]:
import sys

In [ ]:
import warnings

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
sys.path.append('..')

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from sklearn.metrics import auc

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from source.code.models.svdbasedrecommender import SVDBasedRecommender

In [ ]:
from source.code.models.songfrequencybasedrecommender import SongFrequencyBasedRecommender

In [ ]:
from source.code.transformers.metafeaturesextractor import MetaFeaturesExtractor

# Read the data

In [ ]:
data_directory = '../data/datasets/'

## Song extra info

In [ ]:
song_extra_info = pd.read_csv(os.path.join(data_directory, 'song_extra_info.csv'))

In [ ]:
song_extra_info.head()

In [ ]:
song_extra_info.info(verbose=True, null_counts=True)

## Train

In [ ]:
train = pd.read_csv(os.path.join(data_directory, 'train.csv'), engine='python')

In [ ]:
train.head().T

In [ ]:
train.info(verbose=True, null_counts=True)

In [ ]:
train['data_from'] = 'train'

In [ ]:
train.msno.nunique()

In [ ]:
train.song_id.nunique()

In [ ]:
train.source_system_tab.nunique()

In [ ]:
train.source_system_tab.value_counts()

In [ ]:
train.source_screen_name.nunique()

In [ ]:
train.source_screen_name.value_counts()

In [ ]:
train.source_type.nunique()

In [ ]:
train.source_type.value_counts()

In [ ]:
train.fillna('unknown', inplace=True)

In [ ]:
train.info(verbose=True, null_counts=True)

In [ ]:
train.target.value_counts()

## Test

In [ ]:
test = pd.read_csv(os.path.join(data_directory, 'test.csv'), engine='python', index_col=0)

In [ ]:
test.head().T

In [ ]:
test.info(verbose=True, null_counts=True)

In [ ]:
test['target'] = None

In [ ]:
test['data_from'] = 'test'

In [ ]:
test.msno.nunique()

In [ ]:
test.song_id.nunique()

In [ ]:
test.source_system_tab.nunique()

In [ ]:
test.source_system_tab.value_counts()

In [ ]:
test.source_screen_name.nunique()

In [ ]:
test.source_screen_name.value_counts()

In [ ]:
test.source_type.nunique()

In [ ]:
test.source_type.value_counts()

In [ ]:
test.fillna('unknown', inplace=True)

In [ ]:
test.info(verbose=True, null_counts=True)

## Songs

In [ ]:
songs = pd.read_csv(os.path.join(data_directory, 'songs.csv'))

In [ ]:
songs.head(4).T

In [ ]:
songs.info(verbose=True, null_counts=True)

In [ ]:
songs.lyricist.fillna('unknown', inplace=True)

In [ ]:
songs.composer.fillna('unknown', inplace=True)

In [ ]:
songs.genre_ids.fillna('unknown', inplace=True)

In [ ]:
songs.language.fillna(-1, inplace=True)

In [ ]:
songs.language = songs.language.astype(np.int64)

In [ ]:
songs.head(4).T

In [ ]:
songs.info(verbose=True, null_counts=True)

In [ ]:
songs.song_length.nunique()

In [ ]:
songs.genre_ids.nunique()

In [ ]:
songs.artist_name.nunique()

In [ ]:
songs.composer.nunique()

In [ ]:
songs.lyricist.nunique()

In [ ]:
songs.language.nunique()

## Members

In [ ]:
members = pd.read_csv(os.path.join(data_directory, 'members.csv'))

In [ ]:
members.head(4).T

In [ ]:
members.info(verbose=True, null_counts=True)

In [ ]:
members.fillna('unknown', inplace=True)

In [ ]:
members.info(verbose=True, null_counts=True)

In [ ]:
members.head(4).T

In [ ]:
members.registration_init_time = pd.to_datetime(members.registration_init_time, format='%Y%m%d')

In [ ]:
members.expiration_date = pd.to_datetime(members.expiration_date, format='%Y%m%d')

## Sample submission

In [ ]:
sample_submission = pd.read_csv(os.path.join(data_directory, 'sample_submission.csv'))

In [ ]:
sample_submission.head(10)

In [ ]:
sample_submission.info(verbose=True, null_counts=True)

# Solutions

In [ ]:
X, y = train[train.columns[:-2]], train[train.columns[-2]]

In [ ]:
X_for_submission = test[test.columns[:-2]]

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
X_for_submission.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
print(type(X_train), type(y_train))

In [ ]:
print(X_test.shape, y_test.shape)

In [ ]:
print(type(X_test), type(y_test))

## Naive frequency approach

In [ ]:
naive_frequency_recommender = SongFrequencyBasedRecommender()

In [ ]:
print('\t', cross_val_score(naive_frequency_recommender, X_train, y_train, cv=5))

In [ ]:
naive_frequency_recommender.fit(X_train, y_train)

In [ ]:
y_pred = naive_frequency_recommender.predict(X_test)

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_pred)

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example (Naive frequency approach)')
plt.legend(loc="lower right")
plt.show()

## SVD

In [ ]:
svd_recommender = SVDBasedRecommender()

In [ ]:
print('\t', cross_val_score(svd_recommender, X_train, y_train, cv=5))

In [ ]:
svd_recommender.fit(X_train, y_train)

In [ ]:
y_pred = svd_recommender.predict(X_test)

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_pred)

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example (SVD)')
plt.legend(loc="lower right")
plt.show()

## FM

In [ ]:
categorical_features = [
    'source_system_tab',
    'source_screen_name',
    'city',
    'gender'
]

In [ ]:
categorical_features_lang = [
    'language'
]

In [ ]:
numerical_features = [
    'bd',
    'song_length',
    'days_registered'
]

In [ ]:
num_features_pipeline = Pipeline([
    ('impute', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('discretize', KBinsDiscretizer(n_bins=4, encode='onehot-dense'))
])

In [ ]:
cat_features_pipeline = Pipeline([
    ('impute', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

In [ ]:
cat_features_pipeline_lang = Pipeline([
    ('impute', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=-1)),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_features_pipeline, numerical_features),
        ('cat', cat_features_pipeline, categorical_features),
        ('cat_lang', cat_features_pipeline_lang, categorical_features_lang)
    ]
)

In [ ]:
unified_pipeline = Pipeline(
    steps=[
        ('add_meta_info', MetaFeaturesExtractor(user_meta=members, item_meta=songs)),
        ('preprocessing', preprocessor)
    ]
)

In [ ]:
X_train = unified_pipeline.fit_transform(X_train, y_train)

In [ ]:
X_train.shape

In [ ]:
from tffm import TFFMClassifier

In [ ]:
import tensorflow as tf

In [ ]:
model = TFFMClassifier(
    order=6,
    rank=10,
    optimizer=tf.train.AdamOptimizer(learning_rate=0.001),
    n_epochs=100,
    batch_size=-1,
    init_std=0.001,
    input_type='dense'
)

In [ ]:
model.fit(X_train, y_train.values, show_progress=True)

In [ ]:
X_test = unified_pipeline.transform(X_test)

In [ ]:
X_test.shape

In [ ]:
y_pred = model.predict_proba(X_test)

In [ ]:
y_pred.shape

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_pred[:, 1])

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example (SVD)')
plt.legend(loc="lower right")
plt.show()